In [10]:
# import urllib2
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup

def getAllDoxyDonkeyPosts(url,links):
#     request = urllib2.Request(url)
#     response = urllib2.urlopen(request)
    http = urllib3.PoolManager()

#     url = 'http://feeds.bbci.co.uk/news/rss.xml'
    response = http.request('GET', url)
    soup = BeautifulSoup(response, "lxml")
    for a in soup.findAll('a'):
        try:
            url = a['href']
            title = a['title']
            if title == "Older Posts":
                print (title, url)
                links.append(url)
                getAllDoxyDonkeyPosts(url,links)
        except:
            title = ""
    return

blogUrl = "http://doxydonkey.blogspot.in"
links = []
getAllDoxyDonkeyPosts(blogUrl,links)

In [11]:
def getDoxyDonkeyText(testUrl):
    http = urllib3.PoolManager()

    #     url = 'http://feeds.bbci.co.uk/news/rss.xml'
    response = http.request('GET', url)
    soup = BeautifulSoup(response, "lxml")
    mydivs = soup.findAll("div", {"class":'post-body'})
    
    posts =[]
    for div in mydivs:
        posts+=map(lambda p:p.text.encode('ascii', errors='replace').replace("?"," "), div.findAll("li"))
    return posts

In [12]:
doxyDonkeyPosts = []
for link in links:
    doxyDonkeyPosts+=getDoxyDonkeyText(link)

In [13]:
doxyDonkeyPosts

[]

In [41]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')

In [43]:
X = vectorizer.fit_transform(doxyDonkeyPosts)
X

<2465x12428 sparse matrix of type '<type 'numpy.float64'>'
	with 247984 stored elements in Compressed Sparse Row format>

In [44]:
X[0]

<1x12428 sparse matrix of type '<type 'numpy.float64'>'
	with 125 stored elements in Compressed Sparse Row format>

In [45]:
print X[0]

  (0, 738)	0.0700267614388
  (0, 10286)	0.0590460757607
  (0, 4212)	0.0614734464037
  (0, 11180)	0.0552777195907
  (0, 2421)	0.0780548982722
  (0, 11160)	0.0592684209687
  (0, 9723)	0.0559448996494
  (0, 6406)	0.0511721887506
  (0, 678)	0.0506034325452
  (0, 10190)	0.104789906333
  (0, 8144)	0.0951615283425
  (0, 11477)	0.0803047144103
  (0, 7409)	0.0866082133073
  (0, 8717)	0.0753013458579
  (0, 3593)	0.0552777195907
  (0, 3890)	0.118536841937
  (0, 10030)	0.0700267614388
  (0, 3105)	0.0830582668246
  (0, 8402)	0.0830582668246
  (0, 11708)	0.0916115818598
  (0, 8133)	0.0837689595714
  (0, 1625)	0.0719646227776
  (0, 7525)	0.117226110017
  (0, 7241)	0.035646969416
  (0, 2964)	0.0530549941883
  :	:
  (0, 5749)	0.0914234747621
  (0, 90)	0.0493366982297
  (0, 9847)	0.0717513993752
  (0, 3110)	0.0332219620451
  (0, 9957)	0.0556068010977
  (0, 10509)	0.0361534628868
  (0, 12148)	0.0421617908983
  (0, 4813)	0.0464038715942
  (0, 9618)	0.0450563714211
  (0, 11759)	0.0592684209687
  (0, 8280)	

In [46]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)

In [47]:
km.fit(X)

Initialization complete
Iteration  0, inertia 4615.041
Iteration  1, inertia 2359.133
Iteration  2, inertia 2350.761
Iteration  3, inertia 2348.676
Iteration  4, inertia 2347.730
Iteration  5, inertia 2346.988
Iteration  6, inertia 2346.534
Iteration  7, inertia 2346.219
Iteration  8, inertia 2346.034
Iteration  9, inertia 2345.942
Iteration 10, inertia 2345.873
Iteration 11, inertia 2345.804
Iteration 12, inertia 2345.751
Iteration 13, inertia 2345.732
Iteration 14, inertia 2345.712
Iteration 15, inertia 2345.703
Iteration 16, inertia 2345.684
Iteration 17, inertia 2345.665
Iteration 18, inertia 2345.642
Iteration 19, inertia 2345.603
Iteration 20, inertia 2345.577
Iteration 21, inertia 2345.551
Iteration 22, inertia 2345.531
Iteration 23, inertia 2345.514
Iteration 24, inertia 2345.505
Iteration 25, inertia 2345.495
Iteration 26, inertia 2345.476
Iteration 27, inertia 2345.474
Converged at iteration 27


KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=3, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=True)

In [48]:
import numpy as np
np.unique(km.labels_, return_counts=True)

(array([0, 1, 2], dtype=int32), array([1505,  416,  544]))

In [49]:
text={}
for i,cluster in enumerate(km.labels_):
    oneDocument = doxyDonkeyPosts[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument

In [50]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk 

In [51]:
_stopwords = set(stopwords.words('english') + list(punctuation)+["million","billion","year","millions","billions","y/y","'s","''","``"])
 

In [52]:
keywords = {}
counts={}
for cluster in range(3):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100, freq, key=freq.get)
    counts[cluster]=freq

In [53]:
unique_keys={}
for cluster in range(3):   
    other_clusters=list(set(range(3))-set([cluster]))
    keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique=set(keywords[cluster])-keys_other_clusters
    unique_keys[cluster]=nlargest(10, unique, key=counts[cluster].get)

In [54]:
unique_keys

{0: ['amazon',
  'quarter',
  'customers',
  'microsoft',
  'price',
  'profit',
  'cloud',
  'inc.',
  'analysts',
  'chief'],
 1: ['uber',
  'round',
  'capital',
  'raised',
  'funding',
  'valuation',
  'investment',
  'private',
  'lyft',
  'venture'],
 2: ['facebook',
  'twitter',
  'ads',
  'video',
  'ad',
  'search',
  'apps',
  'social',
  'advertising',
  'youtube']}

In [55]:
article = "Facebook Inc. has been giving advertisers an inflated metric for the average time users spent watching a video, a measurement that may have helped boost marketer spending on one of Facebook’s most popular ad products. The company, owner of the world’s largest social network, only counts a video as viewed if it has been seen for more than 3 seconds. The metric it gave advertisers for their average video view time incorporated only the people who had watched the video long enough to count as a view in the first place, inflating the metric because it didn’t count anyone who didn’t watch, or watched for a shorter time. Facebook’s stock fell more than 1.5 percent in extended trading after the miscalculation was earlier reported in the Wall Street Journal. Facebook had disclosed the mistake in a posting on its advertiser help center web page several weeks ago. Big advertising buyers and marketers are upset about the inflated metric, and asked the company for more details, according to the report in the Journal, citing unidentified people familiar with the situation. The Menlo Park, California-based company has kept revenue surging in part because of enthusiasm for its video ads, which advertisers compare in performance to those on Twitter, YouTube and around the web."

In [68]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X,km.labels_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [69]:
test=vectorizer.transform([article.decode('utf8').encode('ascii',errors='ignore')])

In [70]:
test

<1x12428 sparse matrix of type '<type 'numpy.float64'>'
	with 85 stored elements in Compressed Sparse Row format>

In [71]:
classifier.predict(test)

array([0], dtype=int32)